In [1]:
# Additional Dependencies
!pip install barbar torchsummary

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


# Content Based Image Retrieval (CBIR)


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import time
import copy
import pickle
from barbar import Bar
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from PIL import Image
import cv2
%matplotlib inline

import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from torchvision import transforms
from torchsummary import summary

from tqdm import tqdm
from pathlib import Path
import gc
RANDOMSTATE = 0

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
# Find if any accelerator is presented, if yes switch device to use CUDA or else use CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [4]:
# preparing intermediate DataFrame
datasetPath = Path('/kaggle/input/cbir-dataset/dataset/')
df = pd.DataFrame()

df['image'] = [f for f in os.listdir(datasetPath) if os.path.isfile(os.path.join(datasetPath, f))]
df['image'] = '/kaggle/input/cbir-dataset/dataset/' + df['image'].astype(str)

df.head()

,image
0,/kaggle/input/cbir-dataset/dataset/1269.jpg
1,/kaggle/input/cbir-dataset/dataset/3863.jpg
2,/kaggle/input/cbir-dataset/dataset/623.jpg
3,/kaggle/input/cbir-dataset/dataset/2193.jpg
4,/kaggle/input/cbir-dataset/dataset/3750.jpg


In [5]:
print(df.shape)

(4738, 1)


# Data Preparation

In [6]:
class CBIRDataset(Dataset):
    def __init__(self, dataFrame):
        self.dataFrame = dataFrame
        
        self.transformations = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
    
    def __getitem__(self, key):
        if isinstance(key, slice):
            raise NotImplementedError('slicing is not supported')
        
        row = self.dataFrame.iloc[key]
        image = self.transformations(Image.open(row['image']))
        return image
    
    def __len__(self):
        return len(self.dataFrame.index)

In [7]:
# Intermediate Function to process data from the data retrival class
def prepare_data(DF):
    trainDF, validateDF = train_test_split(DF, test_size=0.15, random_state=RANDOMSTATE)
    train_set = CBIRDataset(trainDF)
    validate_set = CBIRDataset(validateDF)
    
    return train_set, validate_set

# AutoEncoder Model

## High Level Structure of an AutoEncoder

![](https://hackernoon.com/hn-images/1*op0VO_QK4vMtCnXtmigDhA.png)

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6, requires_grad=False):
        super(GeM, self).__init__()
        self.p = nn.Parameter(torch.ones(1)*p, requires_grad=requires_grad)
        self.eps = eps

    def forward(self, x):
        return self.gem(x, p=self.p, eps=self.eps)

    def gem(self, x, p=3, eps=1e-6):
#         return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1./p)
        # Clamp the input tensor and raise it to the power 'p'
        x_pow = x.clamp(min=eps).pow(p)

        # Perform average pooling on a 2x2 block
        x_avg_pool = F.avg_pool2d(x_pow, (3, 3), stride = (1,1), padding = (1, 1))

        # Apply the final power operation
        gem_result = x_avg_pool.pow(1.0 / p)

        return gem_result


    def __repr__(self):
        return self.__class__.__name__ + '(' + 'p=' + '{:.4f}'.format(self.p.data.tolist()[0]) + ', ' + 'eps=' + str(self.eps) + ')'

In [9]:
import torch

# Create a sample input tensor
sample_input = torch.randn(1, 256, 16, 16)  # Batch size of 1, 256 channels, 16x16 spatial dimensions

# Create an instance of the GeM layer
gem_layer = GeM(p=3, eps=1e-6)

# Pass the input through the GeM layer
output = gem_layer(sample_input)

# Print the input and output shapes
print("Input shape:", sample_input.shape)
print("Output shape:", output.shape)

Input shape: torch.Size([1, 256, 16, 16])
Output shape: torch.Size([1, 256, 16, 16])


In [10]:
class ConvAutoencoder(nn.Module):
    def __init__(self):
        super(ConvAutoencoder, self).__init__()
        self.encoder = nn.Sequential(# in- ( N,3,512,512)
            
            nn.Conv2d(in_channels=3, 
                      out_channels=16, 
                      kernel_size=(3,3), 
                      stride=3, 
                      padding=1),  # (32,16,171,171)
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=2),  # (N,16,85,85)
            
            nn.Conv2d(in_channels=16, 
                      out_channels=8, 
                      kernel_size=(3,3), 
                      stride=2, 
                      padding=1),  # (N,8,43,43)
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=1)  # (N,8,42,42)
        )
        self.decoder = nn.Sequential(
            
            nn.ConvTranspose2d(in_channels = 8, 
                               out_channels=16, 
                               kernel_size=(3,3), 
                               stride=2),  # (N,16,85,85)
            nn.ReLU(True),
 
            nn.ConvTranspose2d(in_channels=16, 
                               out_channels=8, 
                               kernel_size=(5,5), 
                               stride=3, 
                               padding=1),  # (N,8,255,255)
            nn.ReLU(True),

            nn.ConvTranspose2d(in_channels=8, 
                               out_channels=3, 
                               kernel_size=(6,6), 
                               stride=2, 
                               padding=1),  # (N,3,512,512)
            nn.Tanh()
        )
#         self.gem = GeMLayer()  # Add GeM layer as the final layer of the encoder


    def forward(self, x):
        x = self.encoder(x)
#         x = self.gem(x)
        print("Encoder  : ", x.shape)
        x = self.decoder(x)
        print("Decoder : ", x.shape)
        return x

In [11]:
class ConvAutoencoder_v2(nn.Module):
    def __init__(self):
        super(ConvAutoencoder_v2, self).__init__()
        self.gem = GeM()

        self.encoder = nn.Sequential(# in- (N,3,512,512)
            
            nn.Conv2d(in_channels=3, 
                      out_channels=64, 
                      kernel_size=(3,3), 
                      stride=1, 
                      padding=1),
            nn.ReLU(True),
            nn.Conv2d(in_channels=64, 
                      out_channels=64, 
                      kernel_size=(3,3), 
                      stride=1, 
                      padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=2), 
            
            nn.Conv2d(in_channels=64, 
                      out_channels=128, 
                      kernel_size=(3,3), 
                      stride=2, 
                      padding=1),
            nn.ReLU(True),
            nn.Conv2d(in_channels=128, 
                      out_channels=128, 
                      kernel_size=(3,3), 
                      stride=1, 
                      padding=0), 
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=2), 
            
            nn.Conv2d(in_channels=128, 
                      out_channels=256, 
                      kernel_size=(3,3), 
                      stride=2, 
                      padding=1), 
            nn.ReLU(True),
            nn.Conv2d(in_channels=256, 
                      out_channels=256, 
                      kernel_size=(3,3), 
                      stride=1, 
                      padding=1), 
            nn.ReLU(True),
            nn.Conv2d(in_channels=256, 
                      out_channels=256, 
                      kernel_size=(3,3), 
                      stride=1, 
                      padding=1), 
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=2) 
#             GeM()
        )
        self.decoder = nn.Sequential(
            
            nn.ConvTranspose2d(in_channels = 256, 
                               out_channels=256, 
                               kernel_size=(3,3), 
                               stride=1,
                              padding=1), 
 
            nn.ConvTranspose2d(in_channels=256, 
                               out_channels=256, 
                               kernel_size=(3,3), 
                               stride=1, 
                               padding=1),  
            nn.ReLU(True),

            nn.ConvTranspose2d(in_channels=256, 
                               out_channels=128, 
                               kernel_size=(3,3), 
                               stride=2, 
                               padding=0),  
            
            nn.ConvTranspose2d(in_channels=128, 
                               out_channels=64, 
                               kernel_size=(3,3), 
                               stride=2, 
                               padding=1),  
            nn.ReLU(True),
            nn.ConvTranspose2d(in_channels=64, 
                               out_channels=32, 
                               kernel_size=(3,3), 
                               stride=2, 
                               padding=1), 
            
            nn.ConvTranspose2d(in_channels=32, 
                               out_channels=32, 
                               kernel_size=(3,3), 
                               stride=2, 
                               padding=1),  
            nn.ReLU(True),
            
            nn.ConvTranspose2d(in_channels=32, 
                               out_channels=3, 
                               kernel_size=(4,4), 
                               stride=2, 
                               padding=2),  
            nn.Tanh()
        )

    def forward(self, x):
        x_enc = self.encoder(x)
#         x_enc = self.gem(x_enc)
#         print("Encoder output : ", x.shape)
        x_dec = self.decoder(x_enc)
#         print("Decoder output : ", x.shape)
        return x_enc, x_dec

In [12]:
summary(ConvAutoencoder_v2().to(device),(3,512,512))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 512, 512]           1,792
              ReLU-2         [-1, 64, 512, 512]               0
            Conv2d-3         [-1, 64, 512, 512]          36,928
              ReLU-4         [-1, 64, 512, 512]               0
         MaxPool2d-5         [-1, 64, 256, 256]               0
            Conv2d-6        [-1, 128, 128, 128]          73,856
              ReLU-7        [-1, 128, 128, 128]               0
            Conv2d-8        [-1, 128, 126, 126]         147,584
              ReLU-9        [-1, 128, 126, 126]               0
        MaxPool2d-10          [-1, 128, 63, 63]               0
           Conv2d-11          [-1, 256, 32, 32]         295,168
             ReLU-12          [-1, 256, 32, 32]               0
           Conv2d-13          [-1, 256, 32, 32]         590,080
             ReLU-14          [-1, 256,

# Training Function

In [13]:
def load_ckpt(checkpoint_fpath, model, optimizer):
    
    # load check point
    checkpoint = torch.load(checkpoint_fpath)

    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['model_state_dict'])

    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

    # initialize valid_loss_min from checkpoint to valid_loss_min
    #valid_loss_min = checkpoint['valid_loss_min']

    # return model, optimizer, epoch value, min validation loss 
    return model, optimizer, checkpoint['epoch']

def save_checkpoint(state, filename):
    """Save checkpoint if a new best is achieved"""
    print ("=> Saving a new best")
    torch.save(state, filename)  # save checkpoint
    
def train_model(model,  
                criterion, 
                optimizer, 
                #scheduler, 
                num_epochs):
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = np.inf

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0

            # Iterate over data.
            for idx,inputs in enumerate(Bar(dataloaders[phase])):
                inputs = inputs.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs, output_dec = model(inputs)
                    loss = criterion(output_dec, inputs)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
            #if phase == 'train':
            #    scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]

            print('{} Loss: {:.4f}'.format(
                phase, epoch_loss))

            # deep copy the model
            if phase == 'val' and epoch_loss < best_loss:
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
                save_checkpoint(state={   
                                    'epoch': epoch,
                                    'state_dict': model.state_dict(),
                                    'best_loss': best_loss,
                                    'optimizer_state_dict':optimizer.state_dict()
                                },filename='ckpt_epoch_{}.pt'.format(epoch))

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Loss: {:4f}'.format(best_loss))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, optimizer, epoch_loss

In [14]:
EPOCHS = 50
NUM_BATCHES = 32
RETRAIN = False

train_set, validate_set = prepare_data(DF=df)

dataloaders = {'train': DataLoader(train_set, batch_size=NUM_BATCHES, shuffle=True, num_workers=1) ,
                'val':DataLoader(validate_set, batch_size=NUM_BATCHES, num_workers=1)
                }

dataset_sizes = {'train': len(train_set),'val':len(validate_set)}

model = ConvAutoencoder_v2().to(device)

criterion = nn.MSELoss()
# Observe that all parameters are being optimized
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
learning_rate = 0.05
weight_decay = 0.0001

# Create the SGD optimizer with weight decay
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
# Decay LR by a factor of 0.1 every 7 epochs
#exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [15]:
# If re-training is required:
# Load the old model
if RETRAIN == True:
    # load the saved checkpoint
    model, optimizer, start_epoch = load_ckpt('../input/cbirpretrained/conv_autoencoder.pt', model, optimizer)
    print('Checkpoint Loaded')

In [ ]:
model, optimizer, loss = train_model(model=model, 
                    criterion=criterion, 
                    optimizer=optimizer, 
                    #scheduler=exp_lr_scheduler,
                    num_epochs=EPOCHS)

Epoch 0/50
----------
4027/4027: [===============================>] - ETA 1.4sss
train Loss: 0.1348
711/711: [==============================>.] - ETA 0.8ss
val Loss: 0.0702
=> Saving a new best

Epoch 1/50
----------
4027/4027: [===============================>] - ETA 1.0sss
train Loss: 0.0626
711/711: [==============================>.] - ETA 0.7ss
val Loss: 0.0564
=> Saving a new best

Epoch 2/50
----------
4027/4027: [===============================>] - ETA 1.0sss
train Loss: 0.0531
711/711: [==============================>.] - ETA 0.6ss
val Loss: 0.0475
=> Saving a new best

Epoch 3/50
----------
4027/4027: [===============================>] - ETA 1.0sss
train Loss: 0.0435
711/711: [==============================>.] - ETA 0.8ss
val Loss: 0.0410
=> Saving a new best

Epoch 4/50
----------
4027/4027: [===============================>] - ETA 1.0sss
train Loss: 0.0397
711/711: [==============================>.] - ETA 0.6ss
val Loss: 0.0381
=> Saving a new best

Epoch 5/50
----------
402

In [ ]:
# Save the Trained Model
torch.save({
            'epoch': EPOCHS,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, 'conv_autoencoderv2_5ep.pt')

# Inference

## 1. Indexing

In [ ]:
transformations = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])

In [ ]:
# Load Model in Evaluation phase
model = ConvAutoencoder_v2().to(device)
model.load_state_dict(torch.load('/kaggle/working/conv_autoencoderv2_5ep.pt', map_location=device)['model_state_dict'], strict=False)

model.eval()

In [ ]:
def top_n_local_features( local_features, n = 200 ):
    # Step 1: Compute L2 norms for each feature vector
    norms = np.linalg.norm(local_features, axis=(1, 2))  # Compute norms along the feature dimensions

    # Step 2: Get the indices of the top 'n' features based on L2 norm
#     n = 150  # Replace with the number of top features you want to select
    top_indices = np.argsort(norms)[-n:]

    # Step 3: Select the top 'n' local features
    top_n_local_features = local_features[top_indices]
    
#     print(top_n_local_features.shape)
    return top_n_local_features

In [ ]:
import numpy as np

# Define a sample input as a numpy array
sample_input = np.random.randn( 256, 16, 16)

# Call the top_n_local_features function
output = top_n_local_features(sample_input)

# Print the shape of the output
print("Output shape:", output.shape)

In [ ]:
def get_latent_features(images, transformations):
    n = 200
#     latent_features = np.zeros((4738,256,16,16))
    latent_features = np.zeros((4738,256,16,16))
    top_latent_features = np.zeros((4738,n,16,16))
    #latent_features = np.zeros((4738,8,42,42))
    gem = GeM()
    
    for i,image in enumerate(tqdm(images)):
        
        tensor = transformations(Image.open(image)).to(device)
        latent_features[i] = model.encoder(tensor.unsqueeze(0)).cpu().detach().numpy()
#         print( type(latent_features[i]), latent_features[i].shape )
        # Top N = 150 local feature selection
        top_latent_features[i] = top_n_local_features( latent_features[i], n = n )
        top_latent_features[i] = gem(torch.tensor( top_latent_features[i] ) )
        
    del tensor
    gc.collect()
    return top_latent_features

In [ ]:
images = df.image.values
latent_features = get_latent_features(images, transformations)

In [ ]:
indexes = list(range(0, 4738))
feature_dict = dict(zip(indexes,latent_features))
index_dict = {'indexes':indexes,'features':latent_features}

In [ ]:
# write the data dictionary to disk
#with open('features.pkl', "wb") as f:
#    f.write(pickle.dumps(index_dict))

## 2. Image Retrieval 

In [ ]:
def euclidean(a, b):
    # compute and return the euclidean distance between two vectors
#     print(a.shape, b.shape)
    return np.linalg.norm(a - b)

In [ ]:
from numpy.linalg import norm

def cosine_similarity(a,b):
    # Reshape the arrays into 1D vectors
    a_flat = a.reshape(-1)
    b_flat = b.reshape(-1)

    # Compute the cosine similarity
    cos_sim = np.dot(a_flat, b_flat) / (norm(a_flat) * norm(b_flat))
    return cos_sim
# def cosine_distance(a,b):
#     return scipy.spatial.distance.cosine(a, b)

In [ ]:
def perform_search(queryFeatures, index, maxResults=64):

    results = []
    similarity = []

    for i in range(0, len(index["features"])):
        # compute the euclidean distance between our query features
        # and the features for the current image in our index, then
        # update our results list with a 2-tuple consisting of the
        # computed distance and the index of the image
        d = euclidean(queryFeatures, index["features"][i])
#         d = cosine_distance( queryFeatures, index["features"][i] )
        sim = cosine_similarity( queryFeatures, index["features"][i] )
        results.append((d, i))
        similarity.append((sim, i))
    
#     # sort the results and grab the top ones
# #     results = sorted(results)[:maxResults]
#      # Sort similarity in descending order based on the similarity score
#     similarity = sorted(similarity, key=lambda x: x[0], reverse=True)
#     # Grab the top ones based on maxResults
#     similarity = similarity[:maxResults]
# #     print(results)
# #     print(similarity)
#     # return the list of results
# #     return results
#     return similarity
     # Sort similarity in descending order based on the similarity score
    similarity = sorted(similarity, key=lambda x: x[0], reverse=True)

    # Apply softmax to the similarity scores
    similarity_scores = np.array([item[0] for item in similarity])
    softmax_similarity = np.exp(similarity_scores - np.max(similarity_scores)) / np.sum(np.exp(similarity_scores - np.max(similarity_scores)))

    # Combine the softmax scores with their corresponding indices
    softmax_similarity = [(softmax_similarity[i], similarity[i][1]) for i in range(len(similarity))]

    # Grab the top ones based on maxResults
    softmax_similarity = softmax_similarity[:maxResults]
    
    return softmax_similarity

In [ ]:
def build_montages(image_list, image_shape, montage_shape):

    if len(image_shape) != 2:
        raise Exception('image shape must be list or tuple of length 2 (rows, cols)')
    if len(montage_shape) != 2:
        raise Exception('montage shape must be list or tuple of length 2 (rows, cols)')
    image_montages = []
    # start with black canvas to draw images onto
    montage_image = np.zeros(shape=(image_shape[1] * (montage_shape[1]), image_shape[0] * montage_shape[0], 3),
                          dtype=np.uint8)
    cursor_pos = [0, 0]
    start_new_img = False
    for img in image_list:
        if type(img).__module__ != np.__name__:
            raise Exception('input of type {} is not a valid numpy array'.format(type(img)))
        start_new_img = False
        img = cv2.resize(img, image_shape)
        # draw image to black canvas
        montage_image[cursor_pos[1]:cursor_pos[1] + image_shape[1], cursor_pos[0]:cursor_pos[0] + image_shape[0]] = img
        cursor_pos[0] += image_shape[0]  # increment cursor x position
        if cursor_pos[0] >= montage_shape[0] * image_shape[0]:
            cursor_pos[1] += image_shape[1]  # increment cursor y position
            cursor_pos[0] = 0
            if cursor_pos[1] >= montage_shape[1] * image_shape[1]:
                cursor_pos = [0, 0]
                image_montages.append(montage_image)
                # reset black canvas
                montage_image = np.zeros(shape=(image_shape[1] * (montage_shape[1]), image_shape[0] * montage_shape[0], 3),
                                      dtype=np.uint8)
                start_new_img = True
    if start_new_img is False:
        image_montages.append(montage_image)  # add unfinished montage
    return image_montages

In [ ]:
# take the features for the current image, find all similar
# images in our dataset, and then initialize our list of result
# images
fig, ax = plt.subplots(nrows=2,figsize=(15,15))
queryIdx = 3166# Input Index for which images 
MAX_RESULTS = 10


queryFeatures = latent_features[queryIdx]
print( queryFeatures.shape )
results = perform_search(queryFeatures, index_dict, maxResults=MAX_RESULTS)
imgs = []

# loop over the results
for (d, j) in results:
    img = np.array(Image.open(images[j]))
    print(j)
    imgs.append(img)

# display the query image
ax[0].imshow(np.array(Image.open(images[queryIdx])))

# build a montage from the results and display it
montage = build_montages(imgs, (512, 512), (5, 2))[0]
ax[1].imshow(montage)

# Clustering of Images

In [ ]:
# from sklearn.cluster import KMeans, MiniBatchKMeans
# from scipy.spatial.distance import cdist
# from sklearn.metrics import silhouette_samples, silhouette_score

# import matplotlib.cm as cm
# %matplotlib inline

In [ ]:
# def get_latent_features1D(images, transformations):
    
#     latent_features1d = []
    
#     for i,image in enumerate(tqdm(images)):
#         tensor = transformations(Image.open(image)).to(device)
#         latent_features1d.append(model.encoder(tensor.unsqueeze(0)).cpu().detach().numpy().flatten())
        
#     del tensor
#     gc.collect()
#     return latent_features1d

In [ ]:
# images = df.image.values
# latent_features1d = get_latent_features1D(images, transformations)

In [ ]:
# latent_features1d = np.array(latent_features1d)

In [ ]:
# distortions = [] 
# inertias = [] 
# mapping1 = {} 
# mapping2 = {} 
# K = range(10,11) 
  
# for k in tqdm(K): 
#     #Building and fitting the model 
#     kmeanModel = KMeans(n_clusters=k).fit(latent_features1d)      
      
#     distortions.append(sum(np.min(cdist(latent_features1d, kmeanModel.cluster_centers_, 
#                       'euclidean'),axis=1)) / latent_features1d.shape[0]) 
#     inertias.append(kmeanModel.inertia_) 
  
#     mapping1[k] = sum(np.min(cdist(latent_features1d, kmeanModel.cluster_centers_, 
#                  'euclidean'),axis=1)) / latent_features1d.shape[0] 
#     mapping2[k] = kmeanModel.inertia_ 

In [ ]:
# plt.plot(K, distortions, 'bx-') 
# plt.xlabel('Values of K') 
# plt.ylabel('Distortion') 
# plt.title('The Elbow Method using Distortion') 
# plt.show() 

In [ ]:
# X = np.array(latent_features1d)
# K = range(10, 11) 

# for n_clusters in tqdm(K):
#     # Create a subplot with 1 row and 2 columns
#     fig, (ax1, ax2) = plt.subplots(1, 2)
#     fig.set_size_inches(18, 7)

#     # The 1st subplot is the silhouette plot
#     # The silhouette coefficient can range from -1, 1 but in this example all
#     # lie within [-0.1, 1]
#     ax1.set_xlim([-0.1, 1])
#     # The (n_clusters+1)*10 is for inserting blank space between silhouette
#     # plots of individual clusters, to demarcate them clearly.
#     ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

#     # Initialize the clusterer with n_clusters value and a random generator
#     # seed of 10 for reproducibility.
#     clusterer = KMeans(n_clusters=n_clusters, random_state=RANDOMSTATE)
#     cluster_labels = clusterer.fit_predict(X)

#     # The silhouette_score gives the average value for all the samples.
#     # This gives a perspective into the density and separation of the formed
#     # clusters
#     silhouette_avg = silhouette_score(X, cluster_labels)
#     print("For n_clusters =", n_clusters,
#           "The average silhouette_score is :", silhouette_avg)

#     # Compute the silhouette scores for each sample
#     sample_silhouette_values = silhouette_samples(X, cluster_labels)

#     y_lower = 10
#     for i in range(n_clusters):
#         # Aggregate the silhouette scores for samples belonging to
#         # cluster i, and sort them
#         ith_cluster_silhouette_values = \
#             sample_silhouette_values[cluster_labels == i]

#         ith_cluster_silhouette_values.sort()

#         size_cluster_i = ith_cluster_silhouette_values.shape[0]
#         y_upper = y_lower + size_cluster_i

#         color = cm.nipy_spectral(float(i) / n_clusters)
#         ax1.fill_betweenx(np.arange(y_lower, y_upper),
#                           0, ith_cluster_silhouette_values,
#                           facecolor=color, edgecolor=color, alpha=0.7)

#         # Label the silhouette plots with their cluster numbers at the middle
#         ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

#         # Compute the new y_lower for next plot
#         y_lower = y_upper + 10  # 10 for the 0 samples

#     ax1.set_title("The silhouette plot for the various clusters.")
#     ax1.set_xlabel("The silhouette coefficient values")
#     ax1.set_ylabel("Cluster label")

#     # The vertical line for average silhouette score of all the values
#     ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

#     ax1.set_yticks([])  # Clear the yaxis labels / ticks
#     ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

#     # 2nd Plot showing the actual clusters formed
#     colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
#     ax2.scatter(X[:, 0], X[:, 1], marker='.', s=30, lw=0, alpha=0.7,
#                 c=colors, edgecolor='k')

#     # Labeling the clusters
#     centers = clusterer.cluster_centers_
#     # Draw white circles at cluster centers
#     ax2.scatter(centers[:, 0], centers[:, 1], marker='o',
#                 c="white", alpha=1, s=200, edgecolor='k')

#     for i, c in enumerate(centers):
#         ax2.scatter(c[0], c[1], marker='$%d$' % i, alpha=1,
#                     s=50, edgecolor='k')

#     ax2.set_title("The visualization of the clustered data.")
#     ax2.set_xlabel("Feature space for the 1st feature")
#     ax2.set_ylabel("Feature space for the 2nd feature")

#     plt.suptitle(("Silhouette analysis for KMeans clustering on sample data "
#                   "with n_clusters = %d" % n_clusters),
#                  fontsize=14, fontweight='bold')

# plt.show()